# world size, scale, zoom

in mapbox, `worldSize` is measured in `fake pixel` which means we just protend the number of size is in pixel, not real pixel size of screen. Every tile has a dimension of `512*512(pixels)`, so under such a predefined scenario, we have a world size of `512` under tile `level 0`. and `2048` under `level 2`.

```python
worldSize = math.pow(2, tileLevel) * 512;
```

but we dot calculate worldSize in such a way, instead, we use a independent variable called `scale` or `zoom` to do so.

* `scale` is, as its name defined, used for scale base size of world `512`, if current world size is 2048 means scaled 4 times. (scale==4.0);
* `zoom` is a continuous variable (a float not a integer) kind of like tile level which is a discrete variable (a integer)

nomally, `scale` along with `zoom` has coherence under its logics, we define the world size, then we known its `scale` and `zoom`. OR we get `worldSize` and `scale` given by `zoom`.


In [10]:
import math;
scale = 1.36;
worldSize = 512 * scale;
zoom = math.log2(scale);
print(f"world scale: {scale}, world size: {worldSize}, world zoom: {zoom}");

zoom = 0.7;
scale = math.pow(2, zoom);
worldSize = 512 * scale;
print(f"world zoom: {zoom}, world size: {worldSize}, world scale: {scale}")

world scale: 1.36, world size: 696.32, world zoom: 0.44360665147561484
world zoom: 0.7, world size: 831.7464538687851, world scale: 1.624504792712471


# spaces

model to world and world to camera matrices are simple, they are just affine transformations, the last(bottom) row must be $[0,0,0,1]$

$$M_{model-view} = \begin{bmatrix}
a&&b&&c&&T_x \\
d&&e&&f&&T_y\\
g&&h&&i&&T_z\\
0&&0&&0&&1
\end{bmatrix}$$

## perspective projection matrix and MVP

the form of `perspective projection matrix` is deduced as the following formula. **but please be cautious the matrix is deduced under OpenGL CVV( canonical view volume) with z value ranged from -1 to +1;**

NOTE: WebGPU/Vulkan/DirectX/Metal all have 0 to +1 in range.

$$
M_{pers} =
\begin{bmatrix} 
\frac {2near}{right-left}&0&\frac{left+right}{right-left}&0\\
0&\frac {2near}{top-bottom}&-\frac {bottom+top}{top-bottom}&0\\
0&0&\frac {-(near+far)}{far-near}&\frac {-2near*far}{far-near}\\
0&0&-1&0 
\end{bmatrix}
$$

mapbox uses following code to gen such matrix:
```js
// mapbox perspective projection matrix version:
function perspectiveNO(out, fovy, aspect, near, far) {
    let f = 1 / Math.tan(fovy / 2), nf;
    out[0] = f / aspect;    out[4] = 0; out[08] = 0;     out[12] = 0;
    out[1] = 0;             out[5] = f; out[09] = 0;     out[13] = 0;
    out[2] = 0;             out[6] = 0;
    out[3] = 0;             out[7] = 0; out[11] = -1;    out[15] = 0;
    if (far != null && far !== Infinity) {
        nf = 1 / (near - far);
        out[10] = (far + near) * nf;
        out[14] = 2 * far * near * nf;
    } else {
        out[10] = - 1;
        out[14] = - 2 * near;
    }
    return out;
}
```


MVP matrix:

$$
\begin{split}
M_{mvp} &= M_{pers} \times M_{mode-view} \\ &=
\begin{bmatrix} 
\frac {2near}{right-left}&0&\frac{left+right}{right-left}&0\\\ 
0&\frac {2near}{top-bottom}&-\frac {bottom+top}{top-bottom}&0\\\ 
0&0&\frac {-(near+far)}{far-near}&\frac {-2near*far}{far-near}\\\ 
0&0&-1&0 
\end{bmatrix} \times
\begin{bmatrix}
a&&b&&c&&T_x \\
d&&e&&f&&T_y\\
g&&h&&i&&T_z\\
0&&0&&0&&1
\end{bmatrix} \\&=
\begin{bmatrix}
-/-&&-/-&&-/-&&-/-\\
-/-&&-/-&&-/-&&-/-\\
-/-&&-/-&&-/-&&-/-\\
-g&&-h&&-i&&-Tz
\end{bmatrix}
\end{split}
$$

If we use such matrix to multiply a random vector $P_{0-model} = [x_0,y_0,z_0,w_0]$

w value in clipping space would be: $w_1 = -gx_0-hy_0-iz_0-T_z$

## NDC

so after tranfomation, point $P_0$ now is :

$$
P_{0-clip} = \begin{bmatrix}
x_1 \\ y_1 \\ z_1 \\ w_1
\end{bmatrix};
P_{0-ndc} = \begin{bmatrix}
x_1/w_1 \\ y_1/w_1 \\ z_1/w_1 \\ 1.0
\end{bmatrix}
$$

## screen

$$
M_{screen} = 
\begin{bmatrix}
scale_x && 0.0 && 0.0 && trans_x\\
0.0 && scale_y && 0.0 && trans_y \\
0.0 && 0.0 && 1.0 && 0.0 \\
0 && 0 && 0 && 1.0
\end{bmatrix}
$$

$$
M_{screen} \times P_{0-clip} = 
\begin{bmatrix}
scale_x x_1+w_1 trans_x \\
scale_y y_1+w_1 trans_y \\
z_1 \\
w_1
\end{bmatrix}
$$

$$
M_{screen} \times P_{0-ndc} = 
\begin{bmatrix}
scale_x \frac{x_1}{w_1}+ trans_x \\
scale_y \frac{y_1}{w_1}+ trans_y \\
\frac{z_1}{w_1} \\
1
\end{bmatrix}
$$

as the results illustrated, we do `perspective division` right after clipping is the same as we do it in the vertex shader. 

in mapbox, class `Transform` defines projection matrix called `cameraToClip`, and `worldToCamera` as view matrix, they all recalculated in `_calcMatrices` member method, and mapbox multipled they two into a PV matrix called `projMatrix` which maybe used to get the final MVP matrix of a specific tile.

# labelPlaneMatrix

In `Transform` class, there is a matrix called `labelPlaneMatrix`, if u check its doc, u may know this matrix is "`Inverse of glCoordMatrix, from NDC to screen coordinates, [-1, 1] x [-1, 1] --> [0, w] x [h, 0]`". Yes that's right, but same with `projMatrix` in `Transform` class too, it's just a half-way matrix, the final one should be multiplied by tile's `projMatrix`(a mat cascaded as `MVP` matrix).

so, the final `labelPlaneMatrix` can transform a point in tile space directly to screen space, awesome!

# SymbolBuffers::dynamicLayoutVertexArray (type of StructArray)

Symbol Projection continueously update symbols' placement (layout), the newly calculated positions are all stored in this array with 4 float components as a record `a_projected_pos`, and then upload onto `dynamicLayoutVertexBuffer` which is a instance of `VertexBuffer`;

what should be noticed is: `x`and`y` components in each record represents a coordinate of a position under screen space, and the last (fourth) component illustrates `segment_angle`, which you should take into consideration while programming shader codes. Third value is useless at the moment.


```c
//// pseudo code:
attribute vec4 a_projected_pos;

// offset from pivot pos, in pixels;
attribute vec2 a_offset;

// in radian
highp float segment_angle = a_projected_pos[3]; 
highp float angle_sin = sin(segment_angle);
highp float angle_cos = cos(segment_angle);
mat2 rotation_matrix = mat2(angle_cos, -1.0 * angle_sin, angle_sin, angle_cos);
vec2 offset = rotation_matrix * a_offset;

// pivot position (screen space);
vec4 projected_pos = vec4(a_projected_pos.xy, 0.0, 1.0);

// `u_coord_matrix` can transform vector in screen space to NDC.
gl_Position = u_coord_matrix * vec4(projected_pos.xy + offset, 0.0, 1.0);
```

# all major matrixes in mapbox

* **Transform::projMatrix**: projection from world coordinates (mercator scaled by worldSize) to clip coordinates. equals to `VP` matrix.
* **Transform::glCoordMatrix**: Transform from screen coordinates to GL NDC, `[0,w] x [h,0] --> [-1,1] x [-1,1]`.
* **Transform::labelPlaneMatrix**: Inverse of glCoordMatrix, from NDC to screen space, `[-1, 1] x [-1, 1] --> [0, w] x [h, 0]`.
* temp **posMatrix**: transform from tile space to world space.
* **OverscaledTileID::projMatrix**: equals to `Transform::projMatrix * posMatrix`, the final `MVP` matrix for a specific tile.
* temp **labelPlaneMatrixRendering**: equal to `Transform.labelPlaneMatrix * OverscaledTileID::projMatrix`, the direct matrix transforms position (in tile space with 8192 in extent) to screen space. Please consider `perspective division` issuses.

# terrain

mapbox supports at least two types of terrains: `mapbox` self-defined terrain, and `terrarium`. 

These terrains are all stored within images, to encode(pack) and decode(unpack) values from each other, different types have different formulas:

In [4]:
import math;

# mapbox:
def mapboxEncode(height):
  value = math.floor((height + 10000) * 10);
  r = value >> 16;
  g = value >> 8 & 0x0000FF;
  b = value & 0x0000FF;
  return [r, g, b, 255];

print(f"mapbox encoding: 0 meter encodes to {mapboxEncode(0)}")

color=[1,134,27,255];
def mapboxDecode(c):
    return (c[0]*256*256+c[1]*256+c[2])*.1 - 10000.0;

print(f"mapbox decode: {color} decodes to {mapboxDecode(color)}")

mapbox encoding: 0 meter encodes to [1, 134, 160, 255]
mapbox decode: [1, 134, 27, 255] decodes to -13.299999999999272


In [15]:
# terrarium:
def terrariumEncode(height):
  height += 32768;
  r = math.floor(height / 256.0);
  g = math.floor(height % 256);
  b = math.floor((height - math.floor(height)) * 256.0);
  return [r, g, b];

print(f"terrarium encode: 0 meter encodes to {terrariumEncode(0)}");

color=[1,134,160,255];
def terrariumDecode(c):
    # 32768 == 2 to 15 power
    return (c[0]*256+c[1]+c[2]/256)-32768;  

print(f"terrarium height: {color} decodes to {terrariumDecode(color)}")

terrarium encode: 0 meter encodes to [128, 0, 0]
terrarium height: [1, 134, 160, 255] decodes to -32377.375
